# GRU

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import os
from sklearn import preprocessing
from scikeras.wrappers import KerasRegressor
from keras.models import Sequential
from sklearn.model_selection import PredefinedSplit, RandomizedSearchCV
from keras.layers import Dense, Dropout, GRU
from keras import optimizers, metrics

In [2]:
folder = os.getcwd()
train = pd.read_csv(os.path.join(os.path.dirname(folder), "data/training.csv"))
y_train_tonorm = train.iloc[:, 1]
X_train_tonorm = train.iloc[:, 2:30]

test = pd.read_csv(os.path.join(os.path.dirname(folder), "data/testing.csv"))
y_test_tonorm = test.iloc[:, 1]
X_test_tonorm = test.iloc[:, 2:30]

### Data Preparation

In [3]:
# Normalization
scaler_x_train = preprocessing.MinMaxScaler()
x_train_scaled = scaler_x_train.fit_transform(X_train_tonorm)
x_train = pd.DataFrame(x_train_scaled)
x_train.columns=list(X_train_tonorm.columns)

scaler_y_train = preprocessing.MinMaxScaler()
y_train_scaled = scaler_y_train.fit_transform(y_train_tonorm.values.reshape(-1, 1))
y_train = pd.DataFrame(y_train_scaled)

x_test_scaled = scaler_x_train.transform(X_test_tonorm)
x_test = pd.DataFrame(x_test_scaled)
x_test.columns=list(X_test_tonorm.columns)

scaler_y_test = preprocessing.MinMaxScaler()
y_test_scaled = scaler_y_train.transform(y_test_tonorm.values.reshape(-1, 1))
y_test = pd.DataFrame(y_test_scaled)

### Model Estimation

In [4]:
def create_model(activation,learn_rate,dropout_rate,neurons):
    
    model = Sequential()
    model.add(GRU(neurons, return_sequences=True, input_shape=(1,28), activation=activation))
    model.add(GRU(neurons, return_sequences=True, activation=activation))
    model.add(Dense(neurons, activation=activation))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1, activation='linear'))
    model.compile(loss='mean_squared_error', 
                optimizer=optimizers.Adam(learning_rate=learn_rate), 
                metrics=[metrics.RootMeanSquaredError()])
    return model

activation =  ['relu','selu', 'elu', 'linear', 'tanh'] 
learn_rate = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5]
dropout_rate = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
neurons = [1,5, 10, 20]
epochs = [10, 20, 30, 100, 200]
batch_size = [100, 500, 1000, 1500]

param_grid = dict(activation=activation, learn_rate=learn_rate, dropout_rate=dropout_rate,
                  neurons=neurons, epochs=epochs, batch_size=batch_size)
my_model = KerasRegressor(build_fn=create_model, ctivation=activation, learn_rate=learn_rate, dropout_rate=dropout_rate,
                  neurons=neurons, epochs=epochs, batch_size=batch_size)

In [5]:
execute_cv = False

if execute_cv:
    np.random.seed(42)
    allresults=pd.DataFrame()
    
    for i in range(80,105,5):
        # define X and y segment for CV
        dim_train_val = round(x_train.shape[0]/100*i)
        x_train_val = x_train.iloc[0:dim_train_val]
        x_train_cv = x_train_val.iloc[0:round(x_train_val.shape[0]*0.9)]
        x_val = x_train_val.iloc[round(x_train_val.shape[0]*0.9):x_train_val.shape[0]]
    
        y_train_val = y_train.iloc[0:dim_train_val]
        y_train_cv = y_train_val.iloc[0:round(y_train_val.shape[0]*0.9)]
        y_val = y_train_val.iloc[round(y_train_val.shape[0]*0.9):y_train_val.shape[0]]
    
        split_index = [-1 if x in x_train_cv.index else 0 for x in range(dim_train_val)]
        pds = PredefinedSplit(test_fold = split_index)
    
        x_train_val = np.expand_dims(x_train_val.to_numpy(),axis=1)
        opt = RandomizedSearchCV(my_model, param_distributions=param_grid, cv=pds, scoring='neg_root_mean_squared_error', n_iter=500, random_state=123,
                               n_jobs=-1)
        opt.fit(x_train_val,y_train_val )
        
        res = pd.DataFrame(opt.cv_results_)
        k=res.pivot_table(index=["param_activation","param_batch_size","param_dropout_rate","param_epochs","param_learn_rate", "param_neurons"],
                        values=["mean_test_score"])
        k['%']=i
        print(k)
        allresults = allresults.append(k)

        results =allresults.groupby(['param_activation', 'param_batch_size', 'param_dropout_rate', 'param_epochs',
                             'param_learn_rate', 'param_neurons'])['mean_test_score'].mean()

        results.columns = ['param_activation', 'param_batch_size', 'param_dropout_rate', 'param_epochs',
                            'param_learn_rate','param_neurons']
        results = results=pd.DataFrame(results)
        results = results.reset_index()
        row_index = results['mean_test_score'].idxmax()
        print("Best Parameters")
        print(results.iloc[row_index,:])

else:
    np.random.seed(42)
    def create_model():
        
        model = Sequential()
        model.add(GRU(20, return_sequences=True, input_shape=(1,28), activation='linear'))
        model.add(GRU(20, return_sequences=True, activation='linear'))
        model.add(Dense(20, activation='linear'))
        model.add(Dropout(0))
        model.add(Dense(1, activation='linear'))
        model.compile(loss='mean_squared_error',
                    optimizer=optimizers.Adam(learning_rate=0.005),
                    metrics=[metrics.RootMeanSquaredError()])
        return model


    my_model = KerasRegressor(build_fn=create_model, epochs=100, batch_size=100)
    x_train_2 = np.expand_dims(x_train.to_numpy(),axis=1)
    my_model.fit(x_train_2, y_train)

Epoch 1/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.0783 - root_mean_squared_error: 0.2749
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0138 - root_mean_squared_error: 0.1171
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0076 - root_mean_squared_error: 0.0869
Epoch 4/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0053 - root_mean_squared_error: 0.0728
Epoch 5/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0043 - root_mean_squared_error: 0.0652
Epoch 6/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0038 - root_mean_squared_error: 0.0616
Epoch 7/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0036 - root_mean_squared_error: 0.0597
Epoch 8/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0034 - root_mean_squared_error: 0.0585
Epoch 9/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0036 - root_mean_squared_error: 0.0597
Epoch 10/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0037 - root_mean_squared_error: 0.0605

### Predictions

In [6]:
x_test_2 = np.expand_dims(x_test.to_numpy(),axis=1)
pred = my_model.predict(x_test_2)
pred_2d = pred.reshape(-1, 1)
pred_final = scaler_y_train.inverse_transform(pred_2d)
pred_final_df = pd.concat([pd.DataFrame(pred_final).reset_index(drop=True), test[["date", "wind_prod"]].reset_index(drop=True)], axis=1)
pred_final_df.columns = ("Predicted", "Date", "Observed")
pred_final_df = pred_final_df[["Date", "Predicted", "Observed"]]

if not os.path.isdir(os.path.join(folder, "predictions")):
    os.makedirs(os.path.join(folder, "predictions"))
pred_final_df.to_csv(os.path.join(folder, "predictions/GRU_predictions.csv"), index = False)

2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 431ms/step
